In [1]:
### inputs ###
task = "bart" # rest or bart
atLas = "mmp" # msdl or haox or mmp

In [2]:
import os
%matplotlib inline
import nilearn
import numpy as np
import scipy
import math
from nilearn import datasets
from nilearn.input_data import NiftiMapsMasker
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import nibabel as nib
import openpyxl
import matplotlib.pyplot as plt
import csv
import pandas as pd
# from sklearn.covariance import GraphLassoCV
correlation_measure = ConnectivityMeasure(kind='partial correlation')

In [3]:
# estimator = GraphLassoCV()
atlas_filename = "MMP1_rois.nii"
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                           memory='nilearn_cache', verbose=5) 

In [4]:
## reading the subject's ID's and groups
wb = openpyxl.load_workbook('Subjects1.xlsx')
wb.get_sheet_names()
sheet = wb.get_sheet_by_name('Sheet1')
N = sheet.max_row

c:\users\paria\appdata\local\programs\python\python36\lib\site-packages\openpyxl\reader\worksheet.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
# Data_DIR = "/Volumes/Seagate/UCLA/func/"
Data_DIR = "F://UCLA//func/"
os.chdir(Data_DIR)
os.getcwd()

'F:\\UCLA\\func'

In [ ]:
## this function calculates hitting-time matrix
def hitting_matrix(correlation_matrix):
    L = np.size(correlation_matrix,axis = 0)
    A_matrix = abs(correlation_matrix)
    np.fill_diagonal(A_matrix, 0)
    
    D_matrix = np.zeros((L,L))
    D_inv = np.zeros((L,L))
    D_sqrt = np.zeros((L,L))
    D_sqrt_inv = np.zeros((L,L))
    for i in range(L):
        D_matrix[i,i] = np.sum(A_matrix[i])
        D_inv[i,i] = 1./D_matrix[i,i]
        D_sqrt[i,i] = np.sqrt(D_matrix[i,i])
        D_sqrt_inv[i,i] = 1./D_sqrt[i,i]

    p_matrix = np.dot(D_inv, A_matrix)
    # Graph Laplacian
    eye_matrix = np.eye(L,L)
    eye_P = eye_matrix - p_matrix

    G_Lap = np.dot(D_sqrt,eye_P)
    G_Lap_n = np.dot(G_Lap, D_sqrt_inv)

    # T = p_matrix
    [eig_val, eig_vec] = np.linalg.eigh(G_Lap_n)
    # print eig_val
    H = np.zeros((L,L))
    d = np.sum(D_matrix)
    for i in range(L):
        for j in range(L):
            deg_i = D_matrix[i,i]
            deg_j = D_matrix[j,j]
            for k in range(L):
                if eig_val[k] != min(eig_val):
                    t_i = (eig_vec[i,k]*eig_vec[i,k])/deg_i
                    t_j = (eig_vec[j,k]*eig_vec[j,k])/deg_j
                    t_ij = (eig_vec[i,k]*eig_vec[j,k])/np.sqrt(deg_i*deg_j)
                    H[i,j] = H[i,j] + d*(1./(eig_val[k]))*(t_j-t_ij)

    H = np.transpose(H)
    return H

In [ ]:
## calculating and saving hitting-time matrix for every subject
for i in range(1,N):
    ID = sheet.cell(row=i+1, column=1).value
    sub = str(ID) 
    Data_Name = "sub-" + sub + "_task-" + task + "_bold_space-MNI152NLin2009cAsym_preproc"    
    # set the name of the desired data set here:
    fmri_filenames = Data_Name + "_filt.nii.gz"
#     data_confs = nilearn.image.high_variance_confounds(fmri_filenames, n_confounds=5, 
#                                                        percentile=2.0, detrend=True, mask_img=None)
    tsv_file = "sub-" + sub+ "_task-" + task + "_bold_confounds.tsv"
    df = pd.read_table(tsv_file, sep='\t', header=0)
    [dim1, dim2] = np.shape(df)
    conf_clmns = [row for row in df if row == "WhiteMatter" or row == "X" or row == "Y" or row == "Z" or row == "RotX" or row == "RotY" or row == "RotZ"]
    data_confs = np.zeros((dim1, 4*7))
    cntr = 0
    for item in conf_clmns:
        data_confs[:, cntr] = np.array(df[item])
        data_confs[:, cntr+7] = np.gradient(data_confs[:, cntr])
        data_confs[:, cntr+2*7] = np.gradient(data_confs[:, cntr+8])
        data_confs[:, cntr+3*7] = np.square(data_confs[:, cntr+2*8])
        cntr = cntr + 1
    data_confs = data_confs[5:,:]
    
    time_series = masker.fit_transform(fmri_filenames, confounds = data_confs)
#     estimator.fit(time_series)
#     precision_matrix = estimator.precision_ 

    correlation_matrix = correlation_measure.fit_transform([time_series])[0]  
    np.savetxt("correlation_partial_" + atLas + "_data_28p_" + Data_Name, correlation_matrix, delimiter=",")
    
    L = np.size(correlation_matrix,axis = 0)
    A_matrix = abs(correlation_matrix)
    np.fill_diagonal(A_matrix, 0)
    
    D_matrix = np.zeros((L,L))
    D_inv = np.zeros((L,L))

    for i in range(L):
        D_matrix[i,i] = np.sum(A_matrix[i])
        D_inv[i,i] = 1./D_matrix[i,i]

    p_matrix = np.dot(D_inv, A_matrix)
    np.savetxt("p_partial_" + atLas + "_data_28p_" + Data_Name, p_matrix, delimiter=",")

    H = hitting_matrix(correlation_matrix)
    np.savetxt("hit_time_" + atLas + "_data_28p_filt_partial_" + Data_Name, H, delimiter=",")

[NiftiLabelsMasker.fit_transform] loading data from MMP1_rois.nii
Resampling labels
[Memory]    0.0s, 0.0min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from MMP1_rois.nii
[Memory]    8.3s, 0.1min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.1s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from MMP1_rois.nii
[Memory]   15.8s, 0.3min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from MMP1_rois.nii
[Memory]   18.4s, 0.3min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from MMP1_rois.nii
________________________________________________________________________________
[Memory] Calling nilear